<a href="https://colab.research.google.com/github/Hoangthang017/CS336.L11/blob/master/Do_an_cuoi_ki/main_search_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# clone github
!git clone https://github.com/Hoangthang017/CS336.L11.git
!pip install underthesea

In [21]:
from glob import glob
import pandas as pd
import pickle
import re
import string
import numpy as np
from underthesea import word_tokenize

In [3]:
# unzip data
import zipfile
with zipfile.ZipFile("/content/CS336.L11/Dataset_Football/bong_da_v1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/")
# load dataset
files_path = glob("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/*/*.txt")

In [7]:
# unzip data
import zipfile
with zipfile.ZipFile("/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/tfidf_vector.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/")
# load data from gg drive 
df_tfidf = pd.read_csv('/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/tfidf_vector.csv', index_col=0)

In [10]:
# load vectorizer
vec = open('/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/vectorizer.pk', 'rb')
vectorizer = pickle.load(vec)
vec.close()

In [ ]:
# import sys
# sys.path.insert(1, '/content/CS336.L11/Do_an_cuoi_ki')

In [11]:
# hàm tiền xử lí văn bản
def text_preprocess(document):
  # loại bỏ html nếu có
  document_test = re.sub(r'@\w+', '', document)

  # viết thường tất cả 
  document_test = document_test.lower()

  # bỏ dấu câu 
  document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)

  # thay tern v_league
  document_test = word_tokenize(document_test,"text").replace("v league","v_league")
  
  # xóa bỏ các khoảng trắng thừa 
  document_test = re.sub(r'\s{2,}', ' ', document_test)

  return document_test

In [23]:
def get_similar_articles(q, df, files_path, vectorizer):
  print("Câu truy vấn:", q)
  # tiền xử lí câu truy vấn
  q = [text_preprocess(q)]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  # print(q_vec)

  sim = {}
  # tính toán độ tương đồng
  for i in range(len(files_path)):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  # print('độ dài ', len(sim))

  # sắp xếp độ tương đồng
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # print(type(sim_sorted))

  # số lượng bài viết tìm được
  rank = 10
  now = 0

  # in kết quả truy vấn được
  for k, v in sim_sorted:
    print("Độ tương đồng: ", v)
    print("Tiêu đề: ", files_name[k])
    print(files_path[k])
    now += 1
    if (now == rank):
      break

In [ ]:
# câu truy vấn
q = 'barcelona'
# gọi hàm truy vấn
get_similar_articles(q,df_tfidf,files_path,vectorizer)